In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import math as mt
import time
import sys

sys.path.insert(1, '/rds/general/user/lr4617/home/4th_Year_Project/Final_Year_Project/')

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
from mPE_fn import mPE_
from scipy.spatial import distance
from scipy.stats import entropy
from mPE_ultis import integrand, ubble, array_list, permutation
from util import rolling_mean, probability, probability_v2, get_mPE_matrix
from sklearn.cluster import KMeans

In [ ]:
###################################################################################################################################################
#################################################################### PARAMS #######################################################################
###################################################################################################################################################
n_PC = 3
significance_level = 0.01
decimals = 3
use_all = False
n_clusters_kmeans = 50
unit_length_entropy = 1500

###################################################################################################################################################
################################################################ LOAD TRAJECTORIES ################################################################
###################################################################################################################################################

print('########################## LOADING TRAJECTORIES ##########################')

modes = ['normal', 'drug']
names = []
root_dir = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/'
# load entire high-dimensional trajectories
cnt = 0
lengths = []
for mode in modes:
    trajs = os.listdir(root_dir + mode + '/' )
    for traj_n in trajs:
        names.append(traj_n)
        if traj_n != '.ipynb_checkpoints': 
            # loading entire high-dimensional trajectory
            path = root_dir + mode + '/' + traj_n + '/' + 'trajectories_na/'
            trajectories = os.listdir(path)
            # removing NaN columns
            nan_cols = []
            for i, time_bin in enumerate(trajectories):
                if time_bin != 'behavs' and time_bin != '.ipynb_checkpoints':
                    trajectory = loadmat(path + time_bin)
                    trajectory = trajectory['trajectory'] 
                    for i in range(trajectory.shape[1]):
                        if np.isnan(trajectory[:, i]).all():
                            nan_cols.append(i)
            
            # decide whether to use sub-sampling or not
            nan_cols = np.asarray(nan_cols)
            a = 0
            if use_all:
                sub_sampling = 1
                window = 150
            elif not use_all:
                sub_sampling = 10
                window = 150
            
            # create trajectory
            if nan_cols.size > 0:
                if len(np.where(nan_cols==nan_cols[0])[0])*3 == len(nan_cols):
                    sampled_trajectories = np.zeros( (int((trajectory.shape[0]*len(trajectories))/sub_sampling), trajectory.shape[1]-len(nan_cols)) )
            else:
                sampled_trajectories = np.zeros( (int((trajectory.shape[0]*len(trajectories))/sub_sampling), trajectory.shape[1]) )

            for i, time_bin in enumerate(trajectories):
                if time_bin != 'behavs':
                    trajectory = loadmat(path + time_bin)
                    trajectory = trajectory['trajectory'] 
                    if nan_cols.size > 0:
                        trajectory = np.delete(trajectory, nan_cols, 1)
                    
                    mov_av_traj = rolling_mean(trajectory, window, overlapping=True)
                    idxs = np.arange(0, mov_av_traj.shape[0], sub_sampling)
                    mov_av_traj =  mov_av_traj[idxs, :]
                    
                    idx_2 = i*mov_av_traj.shape[0]
                    sampled_trajectories[idx_2:idx_2+mov_av_traj.shape[0], 0:sampled_trajectories.shape[1]] = mov_av_traj
                    
            # convert nan to number when not it is a sparse recurrence (not an entire column)
            sampled_trajectories = np.nan_to_num(sampled_trajectories)
            lengths.append(sampled_trajectories.shape[0])

            # append trajectory to all trajectories
            if cnt==0:
                rats = sampled_trajectories
            if cnt>0:
                rats = np.concatenate((rats, sampled_trajectories), axis=0)
                
            cnt += 1

            print(sampled_trajectories.shape)
            
name_out = 'rats_' + str(sub_sampling) + '_window_' + str(window) + '.npy'
path_out = '/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/'
np.save(path_out + name_out, rats)

In [26]:
rats = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/rats_sampling_10_window_150.npy')
print(rats.shape)
lengths = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/lengths.npy')

to_be_removed = '.ipynb_checkpoints'

root_dir = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/'
normal_trajs = os.listdir(root_dir + 'normal')
drug_trajs_1 = os.listdir(root_dir + 'caffeine')
drug_trajs_2 = os.listdir(root_dir + 'amphetamine')

if to_be_removed in normal_trajs:
    idx = normal_trajs.index(to_be_removed)
    normal_trajs.pop(idx)
    
if to_be_removed in drug_trajs_1:
    idx = drug_trajs_1drug_trajs_1.index(to_be_removed)
    drug_trajs_1.pop(idx)

if to_be_removed in drug_trajs_2:
    idx = drug_trajs_2.index(to_be_removed)
    drug_trajs_2.pop(idx)

names = normal_trajs + drug_trajs_1 + drug_trajs_2

(6480000, 60)


In [34]:
###################################################################
###################### DMI: TIME-DEPENDENT ########################
###################################################################

DMIs = np.zeros((20,50,3))

modes = ['normal', 'caffeine', 'amphetamine']

block_lengths = [sum(lengths[i*4:(i*4)+4]) for i in range(len(modes))]
modes = ['normal', 'caffeine', 'amphetamine']
sample_size = 100000
dims = np.arange(60)
n_PC = 3
min_lengths = [820800, 348000, 348000]

for i in range(len(block_lengths)):
    
    print(modes[i])
    
    if i == 0:
        signal_mode = rats[0:block_lengths[0], :]
        print(signal_mode.shape)
    else:
        signal_mode = rats[block_lengths[i-1]:block_lengths[i-1]+block_lengths[i], :]

    for ii in range(4):
    
        if ii == 0 :
            signal = signal_mode[0:lengths[(i*4)], :]
        else:
            signal = signal_mode[ lengths[(i*4) + ii -1]: lengths[(i*4) + ii-1] + lengths[(i*4) + ii], :]
            
            if i == 2 and ii == 1:
                prev = signal_mode[lengths[(i*4) + ii - 2]:lengths[(i*4) + ii-2] + lengths[(i*4) + ii -1], :]
                signal[signal.shape[0]:min_lengths[i], :] = prev[signal.shape[0]:min_lengths[i], :]
        
        pca = PCA(n_components=n_PC)
        reduced_traj = pca.fit_transform(signal)
        
        signal = signal[0:min_lengths[i], :]
        reduced_traj = reduced_traj[0:min_lengths[i], :]
        
        if ii == 0:
            rats_ = signal
            rats_red = reduced_traj
        else:
            rats_ = np.concatenate((rats_, signal), axis=0)
            rats_red = np.concatenate((rats_red, reduced_traj), axis=0)
        
    for iii in range(0,4*min_lengths[i],4*sample_size):
    
        red_signal = rats_red[iii:iii+4*sample_size, :]
        [rH, _] = mPE_(red_signal, 3)

        for dim in range(int(len(dims)/3)):
            
            print('dimension = ', dim)
            
            # retrieve marker signal (x,y,z)
            marker_signal = rats_[iii:iii+4*sample_size, dim:dim+3]
            
            print(marker_signal.shape, red_signal.shape)

            # calculate marker signal mPE
            [mH, _] = mPE_(marker_signal, 3)

            # create joint signal for Joint Dynamical Entropy
            joint_signal = np.concatenate((red_signal, marker_signal), axis=1)
            print(joint_signal.shape)
            
            # caluclate joint entropy using joint signal
            [JH, _] = mPE_(joint_signal, 3)

            # calculate dynamical mutual information
            DMI = rH + mH - JH
            
            print(rH, mH, JH)
            
            # fill array
            DMIs[dim, int(iii/4*sample_size), i] = DMI
            
            print(DMI)
            

normal
(3942000, 60)
dimension =  0
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.28514239]] [[8.89670344]]
[[0.2306028]]
dimension =  1
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.23115881]] [[8.83466114]]
[[0.23866151]]
dimension =  2
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.15514827]] [[8.74454337]]
[[0.25276875]]
dimension =  3
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.19810589]] [[8.73889855]]
[[0.30137119]]
dimension =  4
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.2062689]] [[8.74496935]]
[[0.3034634]]
dimension =  5
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.24508819]] [[8.78549502]]
[[0.30175702]]
dimension =  6
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.2450467]] [[8.79514817]]
[[0.29206238]]
dimension =  7
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[5.09313222]] [[8.59465252]]
[[0.34064355]]
dimension =  8
(400000, 3) (400000, 3)
(400000, 6)
[[3.84216385]] [[4.97018159]] [[8.43506467]]

IndexError: index 10000000000 is out of bounds for axis 1 with size 50

In [ ]:
###################################################################
###################### DMI: TIME-INDEPENDENT ######################
###################################################################
DMIs = np.zeros((len(lengths), 20))

for which_traj, length in enumerate(lengths):

    print('TRAJECTORY ', str(which_traj))
    
    # retrieve trajectory from trajectories
    if which_traj == 0:
        idx = 0
    else:
        idx += lengths[which_traj-1]

    traj = rats[idx:idx+length, :]

    print('########################## APPLY PCA ##########################')
    
    # apply PCA to high-d signal to reduce it to "n_PC" dims
    pca = PCA(n_components=n_PC)
    reduced_traj = pca.fit_transform(traj)
    
    # calculate marker signal mPE
    [HH, _]=mPE_(reduced_traj, 3)
    
    for dim in range(0,traj.shape[1],3):
        
        # retrieve marker signal (x,y,z)
        marker_signal = traj[:, dim:dim+3]
        
        # calculate marker signal mPE
        [mH, _] = mPE_(marker_signal, 3)
        
        # create joint signal for Joint Dynamical Entropy
        joint_signal = np.concatenate((reduced_traj, marker_signal), axis=1)
        
        # caluclate joint entropy using joint signal
        [JH, _] = mPE_(joint_signal, 3)
        
        # calculate dynamical mutual information
        DMI = HH + mH - JH
        
        DMIs[which_traj, int(dim/3)] = DMI[0]
        
        print(DMI[0])

path_out = '/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/'
name_out = 'marker_DMI.npy'
np.save(path_out + name_out, DMIs)